## Installation and Imports

In [1]:
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym

## Configure and Run

In [2]:
config = (
    SACConfig()
    .environment(
        env="HalfCheetah-v3",
        normalize_actions=True
    )
    .training(
        q_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        policy_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        tau=0.005,
        target_entropy="auto",
        n_step=1,  # num of SGD steps per batch of data in training step
        train_batch_size=256,
        target_network_update_freq=1,
        replay_buffer_config={"type":"MultiAgentPrioritizedReplayBuffer"},
        num_steps_sampled_before_learning_starts=10_000,
        optimization_config={
          "actor_learning_rate": 0.0003,
          "critic_learning_rate": 0.0003,
          "entropy_learning_rate": 0.0003,
        },
        clip_actions=False
    )
    .rollouts(
        num_rollout_workers=3,
        rollout_fragment_length=1,
    )
    .resources(num_gpus=0)
    .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .reporting(
        min_sample_timesteps_per_iteration=1000,
        metrics_num_episodes_for_smoothing=5
    )
    .framework(framework="torch")
)

In [3]:
wandb_init = dict(
    save_code=True,
    config={
        "env": "HalfCheetah-v3",
        
        "actor_learning_rate": 0.0003,
        "critic_learning_rate": 0.0003,
        "entropy_learning_rate": 0.0003,
        "framework": "torch",
        
        "num_rollout_workers": 3,
        "num_gpu": 0,
        "metrics_num_episodes_for_smoothing": 5
    },
    tags=["local"],
    notes="Test to inspect scaling on Vast.ai",
    name="HalfCheetah_local"
    # job_type=D
    # monitor_gym=
)

In [4]:
tuner = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        name="HalfCheetah_vast_ai",
        stop={"training_iteration": 3_000, "episode_reward_mean": 150},
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
        callbacks=[
                WandbLoggerCallback(project="HalfCheetah", save_checkpoints=True, **wandb_init)
            ],
        local_dir="./results"
        ),
    param_space=config
)

results = tuner.fit()

2023-02-09 23:25:13,054	INFO worker.py:1538 -- Started a local Ray instance.
2023-02-09 23:25:14,515	INFO wandb.py:250 -- Already logged into W&B.


wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(SAC pid=5255) 2023-02-09 23:25:18,698	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=5255) 2023-02-09 23:25:18,895	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(raylet) [2023-02-09 23:25:22,946 E 4994 5039] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_23-25-09_937334_4881 is over 95% full, available space: 1206038528; capacity: 31845081088. Object creation will fail if spilling is required.
(SAC pid=5255) 2023-02-09 23:25:28,387	WARNING util.py:66 -- Install gputil for GPU system monitoring.
(SAC pid=5255) 2023-02-09 23:25:28,669	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method resu

(raylet) [2023-02-09 23:25:32,970 E 4994 5039] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_23-25-09_937334_4881 is over 95% full, available space: 1200185344; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-09 23:25:42,975 E 4994 5039] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_23-25-09_937334_4881 is over 95% full, available space: 1202999296; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-09 23:25:52,982 E 4994 5039] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_23-25-09_937334_4881 is over 95% full, available space: 1201119232; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-09 23:26:02,992 E 4994 5039] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_23-25-09_937334_4881 is over 95% full, available space: 1200787456; capacity: 31845081088. Object creation will fail if spilli